In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

import matplotlib
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('/content/concrete_compressive_strength.csv')
data.head()

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

In [ ]:
datain = data.iloc[:,0:8]
dataou = data.iloc[:,8:]

In [ ]:
datain.head()

In [ ]:
dataou.head()

In [ ]:
datain_tr, datain_te, dataou_tr, dataou_te = train_test_split(datain, dataou, test_size = 0.2, random_state = 42)

In [ ]:
# fun_sigmoid(v = x*W+B)
def fun_sigmoid(v):
  return 1/(1+np.exp(-v))

v = np.random.randn(100,1) * 5
u = fun_sigmoid(v)
plt.plot(v,u,'.')
plt.xlabel('v')
plt.ylabel('u')



In [ ]:
scalerin = MinMaxScaler(feature_range=(0,1))
scalerin.fit(datain_tr)

In [ ]:
datain_tr_calibrated = scalerin.transform(datain_tr)
datain_te_calibrated = scalerin.transform(datain_te)

print(datain_tr_calibrated.min())
print(datain_tr_calibrated.max())
print("      ")
print(datain_te_calibrated.min())
print(datain_te_calibrated.max())

In [ ]:
scalerou = MinMaxScaler(feature_range=(0,1))
scalerou.fit(dataou_tr)

In [ ]:
dataou_tr_calibrated = scalerou.transform(dataou_tr)
dataou_te_calibrated = scalerou.transform(dataou_te)

print(dataou_tr_calibrated.min())
print(dataou_tr_calibrated.max())
print("      ")
print(dataou_te_calibrated.min())
print(dataou_te_calibrated.max())

In [ ]:
# Create Shallow Net

hidden_layers = [100]

model = tf.keras.models.Sequential()

model.add(Dense(hidden_layers[0], activation='relu', input_dim = 8))
model.add(Dense(dataou_tr_calibrated.shape[1], activation = 'linear'))

model.compile(optimizer='adam', loss = "mean_squared_error")
#www.keras.io

In [ ]:
model.summary()

In [ ]:
history = model.fit(datain_tr_calibrated, dataou_tr_calibrated, epochs = 1000, batch_size = 16, verbose = 1, shuffle=True, validation_split=0.01)

In [ ]:
dataes_tr_calibrated = model.predict(datain_tr_calibrated)
dataes_te_calibrated = model.predict(datain_te_calibrated)

In [ ]:
plt.figure(figsize=[5,5])
plt.plot(dataou_tr_calibrated[:,0], dataes_tr_calibrated[:,0], '*', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Training Results | Shallow Net', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)


In [ ]:
plt.figure(figsize=[5,5])
plt.plot(dataou_te_calibrated[:,0], dataes_te_calibrated[:,0], '*', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Testing Results | Shallow Net', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)

In [ ]:
sns.distplot(dataou_tr_calibrated[:,0] - dataes_tr_calibrated[:,0], bins = 25, kde=False)

In [ ]:
sns.distplot(dataou_te_calibrated[:,0] - dataes_te_calibrated[:,0], bins = 25, kde=False)

In [ ]:
plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()

In [ ]:
# Create Deep Net

hidden_layers = [100, 50, 25]

model = tf.keras.models.Sequential()

model.add(Dense(hidden_layers[0], activation='relu', input_dim = 8))
model.add(Dense(hidden_layers[1], activation='relu'))
model.add(Dense(hidden_layers[2], activation='relu'))
model.add(Dense(dataou_tr_calibrated.shape[1], activation = 'linear'))

model.compile(optimizer='adam', loss = "mean_squared_error")
#www.keras.io

In [ ]:
model.summary()

In [ ]:
history = model.fit(datain_tr_calibrated, dataou_tr_calibrated, epochs = 1000, batch_size = 16, verbose = 1, shuffle=True, validation_split=0.01)

In [ ]:
dataes_tr_calibrated = model.predict(datain_tr_calibrated)
dataes_te_calibrated = model.predict(datain_te_calibrated)

plt.figure(figsize=[5,5])
plt.plot(dataou_tr_calibrated[:,0], dataes_tr_calibrated[:,0], '*', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Training Results | Shallow Net', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)

In [ ]:
plt.figure(figsize=[5,5])
plt.plot(dataou_te_calibrated[:,0], dataes_te_calibrated[:,0], '*', markersize = 3)
plt.plot([0,1],[0,1], '-r', linewidth = 2)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.title('Testing Results | Shallow Net', fontsize = 20)
plt.legend(['datapoint','bi-sector'], fontsize = 20)
matplotlib.rc('xtick', labelsize = 20)
matplotlib.rc('ytick', labelsize = 20)

In [ ]:
plt.figure(figsize=[8,8])
plt.plot(history.history['loss'],'--')
plt.plot(history.history['val_loss'],'-')
plt.title('model loss', fontsize = 20)
plt.ylabel('loss', fontsize = 20)
plt.xlabel('epoch', fontsize = 20)
plt.legend(['training loss','validation loss'], fontsize = 20)
plt.show()